In [2]:
import re
import stanza

# Load the desired Stanza model.
nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma,constituency,ner,mwt')

# Annotate the sentence
# doc = nlp("He disdained death and was not bothered by it, challenging the provocations of the imperialist donkeys with full self-control, repeating the words 'There is no God but God and Mohammed is the messenger of God'.")
# doc = nlp("create a new sheet called 'dragon', add a column named 'china' to it")
doc  = nlp("Qais the son of his father")

contents_list = []
brackets_list = []

def replace_and_store(match):
    content = match.group(1).split()[0]
    bracket = match.group(1).split()[1]
    contents_list.append(content)
    brackets_list.append(bracket)

    return '*'


def conll_table(string, lemmas, ner, sent_count):
    # Add newline after closed brackets without any nested brackets inside
    
    stared_string = re.sub(r'\(([^()]+)\)', replace_and_store, string)

    print(brackets_list)
    # replace 'ROOT' with 'TOP'
    stared_string = stared_string.replace('ROOT', 'TOP')

    # remove spaces from stared_string
    stared_string = ''.join(stared_string.split())

    # if '*' is followed by '*', add space between them
    stared_string = re.sub(r'\*(\*)', r'* \1', stared_string)
    # if '**' was found add space between the '**' to be like this '* *'
    stared_string = re.sub(r'\*\*', r'* *', stared_string)
    # if '*' is followed by '(', add space between them
    stared_string = re.sub(r'\*(\()', r'* \1', stared_string)
    # if ')' is followed by '*', add space between them
    stared_string = re.sub(r'\)(\*)', r') \1', stared_string)
    # if ')' is followed by '(', add space between them
    stared_string = re.sub(r'\)(\()', r') \1', stared_string)

    tree1 = stared_string.split()

    print(tree1)

    # construct a table where 1st column is the brackets and 2nd column is the content and 3rd column is the corresponding '*'
    table = []
    c = 0
    for i in range(len(contents_list)):

      table.append(['wb/a2e/00/a2e_0010','0',c,brackets_list[i],contents_list[i], tree1[i], lemmas[i], '-', '-', '-', ner[i],'-'])

      c += 1
      # if bracket == "." reset c to 0
      if brackets_list[i] == ".":
        c = 0
      
    return table

def wrapper(doc):
    # store the constituency parse in a dictionary
    tree_dict = {}
    table_list = []
    lemmas = []
    # fill ner with _ to match the length of lemmas
    ner = []


    for sent in doc.sentences:
        cons = sent.constituency
        tree_dict[sent] = str(cons)
        for token in sent.tokens:
            # check if ner is 'O' then write it else add '-'
            # print(token)
            if token.ner == 'O':
                ner.append('-')
            # elif token.ner == 'S-PERSON':
            #     # if ner is s-person then write '(PERSON)'
            #     ner.append('(PERSON)')
            else:
                ner.append('-')

        for word in sent.words:
            # check if word is verb then write it else add '_'
            if word.upos == "VERB":
                lemmas.append(word.lemma)
            else:
                lemmas.append('-')
            #lemmas.append(word.lemma)
    
    combined_tree = ''
    for sent in doc.sentences:
       print(tree_dict)
       combined_tree += ' ' + ' '.join(tree_dict[sent].split())
    print(combined_tree)

    table_list.append(conll_table(combined_tree, lemmas, ner, len(doc.sentences)))

    return table_list

# def save_to_conll_file(table_list, filename):
#     with open(filename, 'w') as file:
#         file.write('#begin document (wb/a2e/00/a2e_0010); part 000')
#         file.write('\n')
#         for table in table_list:
#             for row in table:
#                 file.write('\t'.join(str(cell) for cell in row) + '\n')
#             file.write('\n')
#         file.write('#end document')

# def save_to_conll_file(table_list, filename):
#     with open(filename, 'w') as file:
#         # Determine the maximum width for each column
#         max_widths = [max(len(str(row[i])) for table in table_list for row in table) for i in range(len(table_list[0][0]))]
        
#         file.write('#begin document (wb/a2e/00/a2e_0010); part 000\n')
#         for table in table_list:
#             for row in table:
#                 # Align each column in a straight line
#                 aligned_row = '\t'.join(str(cell).ljust(width) for cell, width in zip(row, max_widths))
#                 file.write(aligned_row + '\n')
#             file.write('\n')
#         file.write('#end document')

def save_to_conll_file(table_list, filename):
    with open(filename, 'w') as file:
        # Determine the maximum width for each column
        max_widths = [max(len(str(row[i])) for table in table_list for row in table) for i in range(len(table_list[0][0]))]
        
        file.write('#begin document (wb/a2e/00/a2e_0010); part 000\n')
        for table in table_list:
            for row in table:
                # Align each column in a straight line (right-aligned)
                aligned_row = '    '.join(str(cell).rjust(width) for cell, width in zip(row, max_widths))
                file.write(aligned_row + '\n')
            file.write('\n')
        file.write('#end document')


# Example usage:
directory_path = "corpus/data/conll-2012-test/data/english/annotations/wb/a2e/00/"
filename = directory_path + "a2e_0010.gold_conll"

# Save the output to a .gold_conll file
table_list = wrapper(doc)
save_to_conll_file(table_list, filename)


2024-05-15 20:41:30 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-05-15 20:41:31 INFO: Downloaded file to C:\Users\ramy6\stanza_resources\resources.json
2024-05-15 20:41:31 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| ner          | ontonotes-ww-multi_charlm |

2024-05-15 20:41:31 INFO: Using device: cpu
2024-05-15 20:41:31 INFO: Loading: tokenize
2024-05-15 20:41:31 INFO: Loading: mwt
2024-05-15 20:41:31 INFO: Loading: pos
2024-05-15 20:41:32 INFO: Loading: lemma
2024-05-15 20:41:32 INFO: Loading: constituency
202

{[
  {
    "id": 1,
    "text": "Qais",
    "lemma": "Qais",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "start_char": 0,
    "end_char": 4,
    "ner": "B-PERSON",
    "multi_ner": [
      "B-PERSON"
    ]
  },
  {
    "id": 2,
    "text": "the",
    "lemma": "the",
    "upos": "DET",
    "xpos": "DT",
    "feats": "Definite=Def|PronType=Art",
    "start_char": 5,
    "end_char": 8,
    "ner": "I-PERSON",
    "multi_ner": [
      "I-PERSON"
    ]
  },
  {
    "id": 3,
    "text": "son",
    "lemma": "son",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "start_char": 9,
    "end_char": 12,
    "ner": "I-PERSON",
    "multi_ner": [
      "I-PERSON"
    ]
  },
  {
    "id": 4,
    "text": "of",
    "lemma": "of",
    "upos": "ADP",
    "xpos": "IN",
    "start_char": 13,
    "end_char": 15,
    "ner": "I-PERSON",
    "multi_ner": [
      "I-PERSON"
    ]
  },
  {
    "id": 5,
    "text": "his",
    "lemma": "his",
    "upos": "PRON",
    

In [6]:
import stanza

def get_predicate_arguments(sentence):
    # Initialize Stanza pipeline
    stanza.download('en')  # download English model
    nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos,depparse')

    # Process the sentence
    doc = nlp(sentence)

    # Extract predicate arguments
    predicate_arguments = {}
    for sent in doc.sentences:
        for word in sent.words:
            if word.head == 0:  # ROOT word
                predicate = word.lemma
                predicate_arguments[predicate] = {"predicate": predicate, "arguments": []}
                for child in sent.words:
                    if child.head == word.id:
                        predicate_arguments[predicate]["arguments"].append((child.lemma, child.deprel))
    return predicate_arguments

# Example usage
sentence = "The cat sat on the mat."
result = get_predicate_arguments(sentence)
print(result)


c:\Users\ramy6\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-05 13:10:13 INFO: Downloaded file to C:\Users\ramy6\stanza_resources\resources.json
2024-05-05 13:10:13 INFO: Downloading default packages for language: en (English) ...
2024-05-05 13:13:59 INFO: Downloaded file to C:\Users\ramy6\stanza_resources\en\default.zip
2024-05-05 13:14:04 INFO: Finished downloading models and saved to C:\Users\ramy6\stanza_resources
2024-05-05 13:14:04 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-05-05 13:14:04 INFO: Downloaded file to C:\Users\ramy6\stanza_resources\resources.json
2024-05-05 13:14:04 WARNING: Language en packag

{'sit': {'predicate': 'sit', 'arguments': [('cat', 'nsubj'), ('mat', 'obl'), ('.', 'punct')]}}


In [12]:
# get dependecy tree using spacy
import spacy

def get_dependency_tree(sentence):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sentence)
    dependency_tree = ""
    for token in doc:
        dependency_tree += token.text + " " + token.dep_ + "\n"
    return dependency_tree

# Example usage

sentence = "The cat sat on the mat."
dependency_tree = get_dependency_tree(sentence)
print(dependency_tree)

The det
cat nsubj
sat ROOT
on prep
the det
mat pobj
. punct



In [3]:
# given a sentence, create it's gold_conll format using spacy_conll
import spacy_conll
import spacy

def create_gold_conll(sentence):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sentence)
    conll = spacy_conll.spacy2conll(doc)
    return conll

# Example usage
sentence = "The cat sat on the mat."
conll = create_gold_conll(sentence)
print(conll)

NameError: name 'spacy' is not defined